In [1]:
from .pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [4]:
#tf-idf
from pyspark.ml.feature import HashingTF,IDF,Tokenizer

In [5]:
sentenceData = spark.createDataFrame([(0.0,'Hi I heard about Spark'),(0.0,'I wish java could use case classes'),(1.0,'Logistic,regression,models,are,neat')],['id','sentence'])

In [6]:
sentenceData.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|0.0|Hi I heard about ...|
|0.0|I wish java could...|
|1.0|Logistic,regressi...|
+---+--------------------+



In [7]:
tokenizer = Tokenizer(inputCol='sentence',outputCol='words')

In [8]:
words_data = tokenizer.transform(sentenceData)

In [9]:
words_data.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|0.0|Hi I heard about ...|[hi, i, heard, ab...|
|0.0|I wish java could...|[i, wish, java, c...|
|1.0|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+



In [10]:
hashing_tf = HashingTF(inputCol='words',outputCol='rawFeatures')

In [11]:
featurized_data = hashing_tf.transform(words_data)

In [12]:
featurized_data.show()

+---+--------------------+--------------------+--------------------+
| id|            sentence|               words|         rawFeatures|
+---+--------------------+--------------------+--------------------+
|0.0|Hi I heard about ...|[hi, i, heard, ab...|(262144,[24417,49...|
|0.0|I wish java could...|[i, wish, java, c...|(262144,[20719,24...|
|1.0|Logistic,regressi...|[logistic,regress...|(262144,[84234],[...|
+---+--------------------+--------------------+--------------------+



In [13]:
idf = IDF(inputCol='rawFeatures',outputCol='features')

In [15]:
idf_model = idf.fit(featurized_data)

In [16]:
rescaled_data = idf_model.transform(featurized_data)

In [18]:
rescaled_data.select('id','features').show(truncate=False)

+---+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |features                                                                                                                                                                                        |
+---+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0.0|(262144,[24417,49304,73197,91137,234657],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])                                                     |
|0.0|(262144,[20719,24417,55551,116873,147765,162369,192310],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])|
|1.0|

In [19]:
from pyspark.ml.feature import CountVectorizer

In [20]:
df =  spark.createDataFrame([(0,"a b c".split(" ")),(1,"a b b c a".split(" "))],['id','words'])

In [21]:
df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [22]:
cv = CountVectorizer(inputCol='words',outputCol='features',vocabSize=3,minDF=2.0)

In [24]:
model = cv.fit(df)

In [25]:
result = model.transform(df)

In [26]:
result.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

